# Re-Imaging Price Trends - 2-Way Data Comparison

**Purpose**: Train models on 2 different data versions for performance comparison:
1. **original_author**: 논문 저자의 원본 데이터 (img_data/)
2. **reconstructed**: 우리가 재구성한 데이터 (img_data_reconstructed/)

**Prerequisites**: Both datasets generated and available

In [ ]:
# Environment setup
!pip install -r requirements.txt

from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/ReImaging_Price_Trends')
print(f"Current directory: {os.getcwd()}")

# GPU check
import torch
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

# GPU memory monitoring
def check_gpu_memory():
    if torch.cuda.is_available():
        device = torch.cuda.current_device()
        total_memory = torch.cuda.get_device_properties(device).total_memory
        allocated = torch.cuda.memory_allocated(device)
        print(f"GPU memory: {allocated/1024**3:.1f}/{total_memory/1024**3:.1f}GB ({allocated/total_memory*100:.1f}%)")
        return allocated, total_memory
    else:
        print("GPU unavailable")
        return 0, 0

def cleanup_memory():
    import gc
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
    gc.collect()
    print("Memory cleaned")

check_gpu_memory()

In [ ]:
# =============================================================================
# 2-WAY COMPARATIVE TRAINING CONFIGURATION
# =============================================================================

# Training method
USE_ENSEMBLE = True    # True: ensemble (paper method), False: single model
ENSEMBLE_RUNS = 5      # Number of ensemble models

print(f"=== 2-WAY COMPARATIVE TRAINING SETUP ===")
print(f"Training method: {'Ensemble (' + str(ENSEMBLE_RUNS) + ' models)' if USE_ENSEMBLE else 'Single model'}")
print(f"Data versions: 2-way comparison")
print(f"  1. original_author  (img_data/)")
print(f"  2. reconstructed    (img_data_reconstructed/)")
print(f"")
print(f"Output models:")
print(f"  - CNN20d_I20R20_original_author_run1-5.tar")
print(f"  - CNN20d_I20R20_reconstructed_run1-5.tar")
print(f"")
print(f"🎯 Purpose: Compare original vs reconstructed data performance!")

In [ ]:
# Check both data versions availability
import os

def check_data_version(version_name, data_dir):
    required_dirs = ['weekly_5d', 'monthly_20d', 'quarterly_60d']
    all_ready = True
    
    print(f"\n{version_name.upper()} DATA ({data_dir}):")
    for dir_name in required_dirs:
        dir_path = os.path.join(data_dir, dir_name)
        if os.path.exists(dir_path):
            dat_files = len([f for f in os.listdir(dir_path) if f.endswith('.dat')])
            feather_files = len([f for f in os.listdir(dir_path) if f.endswith('.feather')])
            print(f"  ✓ {dir_name}: {dat_files} .dat, {feather_files} .feather files")
        else:
            print(f"  ✗ {dir_name}: MISSING")
            all_ready = False
    
    return all_ready

# Check both versions
original_author_ready = check_data_version('ORIGINAL AUTHOR', 'img_data')
reconstructed_ready = check_data_version('RECONSTRUCTED', 'img_data_reconstructed') 

print(f"\n=== DATA AVAILABILITY SUMMARY ===")
print(f"Original Author: {'✅ Ready' if original_author_ready else '❌ Missing'}")
print(f"Reconstructed:   {'✅ Ready' if reconstructed_ready else '❌ Missing'}")

ready_count = sum([original_author_ready, reconstructed_ready])
if ready_count == 2:
    print(f"\n🎉 Both datasets ready for comparative training!")
elif ready_count > 0:
    print(f"\n⚠️  Only {ready_count}/2 datasets available. Generate missing datasets first.")
else:
    print(f"\n❌ No datasets available. Run data generation first.")

# Set training flags
CAN_TRAIN_ORIGINAL_AUTHOR = original_author_ready
CAN_TRAIN_RECONSTRUCTED = reconstructed_ready

# CNN5d Training (Original vs Reconstructed Data)

In [ ]:
# CNN5d Training - Original Author Data
print(f"=== CNN5d Training - ORIGINAL AUTHOR Data ===")
print(f"Method: {'Ensemble' if USE_ENSEMBLE else 'Single model'}")
print(f"Output: CNN5d_I5R5_original_author{'_run1-5' if USE_ENSEMBLE else ''}.tar")

if CAN_TRAIN_ORIGINAL_AUTHOR:
    check_gpu_memory()
    
    if USE_ENSEMBLE:
        print(f"\nTraining ensemble ({ENSEMBLE_RUNS} models) on original author data...")
        !python train.py --model CNN5d --image_days 5 --pred_days 5 --ensemble --ensemble_runs {ENSEMBLE_RUNS} --use_original_format --data_version original_author
    else:
        print(f"\nTraining single model on original author data...")
        !python train.py --model CNN5d --image_days 5 --pred_days 5 --use_original_format --data_version original_author
    
    print("✅ CNN5d (original author) training completed")
else:
    print("❌ Original author data not available - skipping")

cleanup_memory()

In [ ]:
# CNN5d Training - Reconstructed Data
print(f"=== CNN5d Training - RECONSTRUCTED Data ===")
print(f"Method: {'Ensemble' if USE_ENSEMBLE else 'Single model'}")
print(f"Output: CNN5d_I5R5_reconstructed{'_run1-5' if USE_ENSEMBLE else ''}.tar")

if CAN_TRAIN_RECONSTRUCTED:
    check_gpu_memory()
    
    if USE_ENSEMBLE:
        print(f"\nTraining ensemble ({ENSEMBLE_RUNS} models) on reconstructed data...")
        !python train.py --model CNN5d --image_days 5 --pred_days 5 --ensemble --ensemble_runs {ENSEMBLE_RUNS} --use_original_format --data_version reconstructed
    else:
        print(f"\nTraining single model on reconstructed data...")
        !python train.py --model CNN5d --image_days 5 --pred_days 5 --use_original_format --data_version reconstructed
    
    print("✅ CNN5d (reconstructed) training completed")
else:
    print("❌ Reconstructed data not available - skipping")

cleanup_memory()

# CNN20d Training (Original vs Reconstructed Data)

In [ ]:
# CNN20d Training - Original Author Data
print(f"=== CNN20d Training - ORIGINAL AUTHOR Data ===")
print(f"Method: {'Ensemble' if USE_ENSEMBLE else 'Single model'}")
print(f"Output: CNN20d_I20R20_original_author{'_run1-5' if USE_ENSEMBLE else ''}.tar")

if CAN_TRAIN_ORIGINAL_AUTHOR:
    check_gpu_memory()
    
    if USE_ENSEMBLE:
        print(f"\nTraining ensemble ({ENSEMBLE_RUNS} models) on original author data...")
        !python train.py --model CNN20d --image_days 20 --pred_days 20 --ensemble --ensemble_runs {ENSEMBLE_RUNS} --use_original_format --data_version original_author
    else:
        print(f"\nTraining single model on original author data...")
        !python train.py --model CNN20d --image_days 20 --pred_days 20 --use_original_format --data_version original_author
    
    print("✅ CNN20d (original author) training completed")
else:
    print("❌ Original author data not available - skipping")

cleanup_memory()

In [ ]:
# CNN20d Training - Reconstructed Data
print(f"=== CNN20d Training - RECONSTRUCTED Data ===")
print(f"Method: {'Ensemble' if USE_ENSEMBLE else 'Single model'}")
print(f"Output: CNN20d_I20R20_reconstructed{'_run1-5' if USE_ENSEMBLE else ''}.tar")

if CAN_TRAIN_RECONSTRUCTED:
    check_gpu_memory()
    
    if USE_ENSEMBLE:
        print(f"\nTraining ensemble ({ENSEMBLE_RUNS} models) on reconstructed data...")
        !python train.py --model CNN20d --image_days 20 --pred_days 20 --ensemble --ensemble_runs {ENSEMBLE_RUNS} --use_original_format --data_version reconstructed
    else:
        print(f"\nTraining single model on reconstructed data...")
        !python train.py --model CNN20d --image_days 20 --pred_days 20 --use_original_format --data_version reconstructed
    
    print("✅ CNN20d (reconstructed) training completed")
else:
    print("❌ Reconstructed data not available - skipping")

cleanup_memory()

# CNN60d Training (Original vs Reconstructed Data)

In [ ]:
# CNN60d Training - Original Author Data
print(f"=== CNN60d Training - ORIGINAL AUTHOR Data ===")
print(f"Method: {'Ensemble' if USE_ENSEMBLE else 'Single model'}")
print(f"Output: CNN60d_I60R60_original_author{'_run1-5' if USE_ENSEMBLE else ''}.tar")
print(f"Warning: Most memory intensive!")

if CAN_TRAIN_ORIGINAL_AUTHOR:
    check_gpu_memory()
    
    if USE_ENSEMBLE:
        print(f"\nTraining ensemble ({ENSEMBLE_RUNS} models) on original author data...")
        !python train.py --model CNN60d --image_days 60 --pred_days 60 --ensemble --ensemble_runs {ENSEMBLE_RUNS} --use_original_format --data_version original_author --batch_size 64
    else:
        print(f"\nTraining single model on original author data...")
        !python train.py --model CNN60d --image_days 60 --pred_days 60 --use_original_format --data_version original_author --batch_size 64
    
    print("✅ CNN60d (original author) training completed")
else:
    print("❌ Original author data not available - skipping")

cleanup_memory()

In [ ]:
# CNN60d Training - Reconstructed Data
print(f"=== CNN60d Training - RECONSTRUCTED Data ===")
print(f"Method: {'Ensemble' if USE_ENSEMBLE else 'Single model'}")
print(f"Output: CNN60d_I60R60_reconstructed{'_run1-5' if USE_ENSEMBLE else ''}.tar")
print(f"Warning: Most memory intensive!")

if CAN_TRAIN_RECONSTRUCTED:
    check_gpu_memory()
    
    if USE_ENSEMBLE:
        print(f"\nTraining ensemble ({ENSEMBLE_RUNS} models) on reconstructed data...")
        !python train.py --model CNN60d --image_days 60 --pred_days 60 --ensemble --ensemble_runs {ENSEMBLE_RUNS} --use_original_format --data_version reconstructed --batch_size 64
    else:
        print(f"\nTraining single model on reconstructed data...")
        !python train.py --model CNN60d --image_days 60 --pred_days 60 --use_original_format --data_version reconstructed --batch_size 64
    
    print("✅ CNN60d (reconstructed) training completed")
else:
    print("❌ Reconstructed data not available - skipping")

cleanup_memory()

# Comparative Portfolio Evaluation

In [ ]:
# CNN20d 2-Way Comparative Evaluation
print(f"=== CNN20d 2-WAY COMPARATIVE EVALUATION ===")
print(f"Strategy: Monthly (I20/R20)")
print(f"Paper benchmark: Sharpe = 2.16")
print(f"🎯 Testing which dataset gives BEST performance!")
print(f"")

ensemble_flag = "--ensemble" if USE_ENSEMBLE else ""

# Original Author Data
if CAN_TRAIN_ORIGINAL_AUTHOR:
    print("📊 CNN20d - Original Author Data:")
    !python test.py --model CNN20d --image_days 20 --pred_days 20 {ensemble_flag} --use_original_format --data_version original_author
    cleanup_memory()
else:
    print("❌ CNN20d original author model not available")

print("\n" + "="*60)

# Reconstructed Data
if CAN_TRAIN_RECONSTRUCTED:
    print("📊 CNN20d - Reconstructed Data:")
    !python test.py --model CNN20d --image_days 20 --pred_days 20 {ensemble_flag} --use_original_format --data_version reconstructed
    cleanup_memory()
else:
    print("❌ CNN20d reconstructed model not available")

print("\n✅ CNN20d 2-way comparative evaluation completed")

# 2-Way Comparative Results Analysis

In [ ]:
import json
import pandas as pd

print("=" * 90)
print("🏆 2-WAY COMPARATIVE RESULTS ANALYSIS")
print("=" * 90)

def load_result(model, data_version, ensemble=USE_ENSEMBLE):
    """Load performance result from JSON file"""
    eval_type = "ensemble" if ensemble else "single"
    
    # Model naming based on data version
    if data_version == 'original_author':
        suffix = "_original_author"
    else:  # 'reconstructed'
        suffix = "_reconstructed"
    
    result_file = f"results/{model}_I{model[3:]}R{model[3:]}{suffix}_{eval_type}_performance.json"
    
    try:
        with open(result_file, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        return None

# Focus on CNN20d for detailed comparison
print(f"\n{'='*90}")
print(f"🎯 CNN20d DETAILED COMPARISON - Monthly Strategy (I20/R20)")
print(f"📊 Paper benchmark: Sharpe = 2.16")
print(f"{'='*90}")

# Load both results
data_versions = ['original_author', 'reconstructed']
results = {}

for version in data_versions:
    results[version] = load_result('CNN20d', version)

# Create comparison table
print(f"\n{'Dataset':<18}{'Sharpe':<10}{'Return':<12}{'Vol':<10}{'Turnover':<12}{'Status':<12}")
print(f"{'-'*80}")

best_sharpe = -999
best_version = None
paper_benchmark = 2.16

for version in data_versions:
    result = results[version]
    
    if result:
        sharpe = result['ls_sharpe_ratio']
        annual_return = result['ls_annual_return'] * 100
        annual_vol = result['ls_annual_vol'] * 100  
        turnover = result['monthly_turnover']
        
        # Compare to paper
        vs_paper = sharpe - paper_benchmark
        status = f"📈 {vs_paper:+.1f}" if vs_paper > 0 else f"📉 {vs_paper:+.1f}"
        
        if sharpe > best_sharpe:
            best_sharpe = sharpe
            best_version = version
        
        print(f"{version:<18}{sharpe:<10.2f}{annual_return:<12.1f}%{annual_vol:<10.1f}%{turnover:<12.1f}%{status:<12}")
        
        # Detailed metrics for each version
        print(f"  └─ Details: L={result['long_sharpe_ratio']:.2f}, S={result['short_sharpe_ratio']:.2f}")
        
    else:
        print(f"{version:<18}{'N/A':<10}{'N/A':<12}{'N/A':<10}{'N/A':<12}{'❌ Missing':<12}")

# Winner analysis
print(f"\n🏆 PERFORMANCE RANKING:")
valid_results = [(v, results[v]['ls_sharpe_ratio']) for v in data_versions if results[v] is not None]
valid_results.sort(key=lambda x: x[1], reverse=True)

for i, (version, sharpe) in enumerate(valid_results):
    medal = "🥇" if i == 0 else "🥈"
    print(f"{medal} {i+1}. {version}: {sharpe:.2f} Sharpe")

if best_version:
    print(f"\n🎉 WINNER: {best_version.upper()} (Sharpe: {best_sharpe:.2f})")
    
    # Compare to baseline
    if len(valid_results) > 1:
        baseline_version, baseline_sharpe = valid_results[-1]  # Worse performer
        improvement = best_sharpe - baseline_sharpe
        improvement_pct = improvement / abs(baseline_sharpe) * 100 if baseline_sharpe != 0 else 0
        print(f"📈 Improvement: {improvement:+.2f} Sharpe ({improvement_pct:+.1f}%)")
    
    # Compare to paper benchmark  
    vs_paper_pct = (best_sharpe - paper_benchmark) / paper_benchmark * 100
    if best_sharpe > paper_benchmark:
        print(f"🚀 BEATS paper benchmark by {best_sharpe - paper_benchmark:.2f} ({vs_paper_pct:+.1f}%)")
    else:
        print(f"📉 Below paper benchmark by {paper_benchmark - best_sharpe:.2f} ({vs_paper_pct:.1f}%)")

# Key insights
print(f"\n💡 KEY INSIGHTS:")

if len(valid_results) == 2:
    best_version, best_sharpe = valid_results[0]
    second_version, second_sharpe = valid_results[1]
    
    print(f"   🥇 Best dataset: {best_version}")
    print(f"   📊 Performance gap: {best_sharpe - second_sharpe:.2f} Sharpe")
    
    # Data quality impact
    if 'original_author' in [v[0] for v in valid_results]:
        original_sharpe = next(sharpe for v, sharpe in valid_results if v == 'original_author')
        reconstructed_sharpe = next(sharpe for v, sharpe in valid_results if v == 'reconstructed')
        
        if reconstructed_sharpe > original_sharpe:
            print(f"   🎯 Our reconstruction IMPROVED performance by {reconstructed_sharpe - original_sharpe:.2f} Sharpe!")
        else:
            print(f"   ⚠️  Original author data still better by {original_sharpe - reconstructed_sharpe:.2f} Sharpe")
            print(f"   🔍 Need to investigate reconstruction differences")

print(f"\n📁 Result files saved in: results/")
if os.path.exists('results'):
    result_files = [f for f in os.listdir('results') if f.endswith('.json') and 'CNN20d' in f]
    for f in sorted(result_files):
        print(f"   📊 {f}")